In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def write_file(path, html_string):
    with open(path, 'w') as f:
        f.write(html_string)
        f.close()

## Research Question: Is there a relationship between property turnover and profit from 2015-2021?

### Hypothesis
I believe that a turnover causes higher maintenace for that year and would hurt the profit for a turnover year. Although the rent would be moved up to market rent when it is rerented, tenants usually have the highest amount maintence request during the first 6 months of tenancy and it cools down as they settle into the house.

### Facts:
#### Bright Leaf
 - Purchase 1/2013
 - Purchase Price: 109k
 - Assumed value: $395,900 - from Zillow
 - Rent raised between 3-5% each year
 - 1 turn over in 2019 but new tenant was old tenant's sister, so rent was not pushed to market rent
 
#### Fort Pike
 - Purchase 2/2013
 - Purchase Price: 187k
 - Assumed value: $527,500 - from Zillow
 - Rent raised between 3-5% each year if tenant renewed
 - 3 turn overs in the period of 2015-2022 and each time rent was listed for market rent (2018, 2020, 2021)

#### How does a move out affect profit?
 - Each move out requires cleaning and repairing the home for a new tenant
 - More expenses incurr because the time it takes to repair home and get it rerented out takes about a month
 - Also have to pay administrative fees to property management to find new tenants
 - New tenants usually have a lot of maintenance request the first 6 months of tenancy because they find broken things that the old tenant lived with

## Cleaning and Preprocessing

In [206]:
data = pd.read_csv("./data/data.csv")

In [207]:
data.head()

,Unnamed: 0,Type,Date,Num,Name,Memo,Clr,Split,Amount,Balance
0,CHASE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Check,12/17/14,NaN,FRANCHISE TAX BOARD,2014 chase chk #444,X,LLC Tax,-800.0,-800.0
2,NaN,Deposit,12/30/14,NaN,"GoldenWest Management, Inc",BRIGHT LEAF 2014,X,Rental Income,12540.0,11740.0
3,NaN,Check,12/30/14,NaN,"GoldenWest Management, Inc",BRIGHT LEAF 2014,X,Management Fees,-900.0,10840.0
4,NaN,Check,12/30/14,NaN,State Farm Fire and Casualty Company,BRIGHT LEAF 2014,X,Insurance Expense,-532.0,10308.0


In [208]:
data["Date"] = pd.to_datetime(data["Date"])

In [209]:
data["Year"] = (data["Date"].apply(lambda x: x.year)) #Extracting Year from transaction date

In [210]:
df = data.loc[data["Year"].isin(np.arange(2015, 2021+1))] #Filtering to have data from 2015-2021
df = df.loc[df["Type"].isin(["Deposit", "Check"])]

In [211]:
df["is_bright_leaf"] = df["Memo"].str.lower().str.contains("bright|leaf") #Labeling which property is which
df["is_fort_pike"] = df["Memo"].str.lower().str.contains("fort|pike")

In [212]:
def selector(row):
    if row["is_bright_leaf"]:
        return "bright leaf"
    elif row["is_fort_pike"]:
        return "fort pike"
    else:
        return None

In [213]:
df["Property"] = df.apply(selector, axis=1)

In [214]:
df.drop(["Unnamed: 0", "Date", "Num", "Clr", "Balance", "Memo", "is_bright_leaf", "is_fort_pike"], axis=1, inplace=True)

In [215]:
#Removing transactions that are not normal maintenance as defined by the Property Management.
#They say that anything over 1k is non-normal.
outlier = df.loc[(df["Amount"]<-1000) | (df["Amount"]>0) & (df["Split"]=="Repairs and Maintenance")].index
df = df.drop(outlier)

## EDA

### Comparing revenue of each property from 2015-2021

In [216]:
income = df.loc[df["Split"]=="Rental Income"].groupby(["Property", "Year"]).sum().reset_index()

In [217]:
income["Property"] = income["Property"].str.title()
income.sort_values("Property", axis=0, ascending=False, inplace=True)

In [218]:
fig = px.line(income, x="Year", y="Amount", color="Property", title="Revenue", color_discrete_map={
        'Fort Pike': 'blue',
        'Bright Leaf': 'green'
    })

In [219]:
fig

We can see that on the years where is turnover, there tends to be a dip depending on how long it takes to rerent it back out. Overall though, we can see that the revenue for Fort Pike is much higher than Bright Leaf

In [220]:
write_file("./visuals/Revenue.html", fig.to_html())

### Repair cost distribution

In [221]:
repairs = df[df["Split"]=="Repairs and Maintenance"].copy()
repairs["Amount"] = repairs["Amount"].apply(lambda x: x*-1)
repairs.head()

,Type,Name,Split,Amount,Year,Property
29,Check,MERIDIAN PROPERTY SERVICES,Repairs and Maintenance,360.0,2015.0,fort pike
34,Check,Cortis Property service,Repairs and Maintenance,225.0,2015.0,bright leaf
52,Check,Cortis Property service,Repairs and Maintenance,195.0,2015.0,bright leaf
68,Check,Cortis Property service,Repairs and Maintenance,237.0,2015.0,bright leaf
89,Check,"GoldenWest Management, Inc",Repairs and Maintenance,125.0,2016.0,bright leaf


In [222]:
fig = px.box(repairs, x="Property", y="Amount", color="Property", title="Repair Cost Distribution by Property")
fig

This shows that the expenses for Fort Pike are positively skewed which means that there are frequent smaller maintenace expenses and a few large maintence expenses. Bright Leaf is more normally distributed and is a little below fort pike which means the expenses tend to be less.

In [223]:
write_file("./visuals/repair_dist.html", fig.to_html())

### Number of maintenance request each year

In [224]:
maintenance_year = repairs.groupby("Year")["Property"].value_counts().rename("count").reset_index()
maintenance_year

,Year,Property,count
0,2015.0,bright leaf,3
1,2015.0,fort pike,1
2,2016.0,fort pike,5
3,2016.0,bright leaf,3
4,2017.0,bright leaf,3
5,2017.0,fort pike,2
6,2018.0,fort pike,5
7,2018.0,bright leaf,1
8,2019.0,bright leaf,2
9,2019.0,fort pike,1


In [225]:
fig = make_subplots(
    rows=1, cols=1)
#Fort Pike Graph
FP_maintenance = maintenance_year[maintenance_year["Property"]=="fort pike"]
trace1 = go.Bar(x = FP_maintenance["Year"], y=FP_maintenance["count"], marker=dict(color="Blue", opacity=1), name="Fort Pike")
fig.add_trace(trace1)

#Bright Leaf Graph
BL_maintenance = maintenance_year[maintenance_year["Property"]=="bright leaf"]
trace2 = go.Bar(x = BL_maintenance["Year"], y=BL_maintenance["count"], marker=dict(color="green", opacity=1), name="Bright Leaf")
fig.add_trace(trace2)

fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Amount")
fig.update_layout(title = "Number of Maintenance Request by Year", barmode="group")

Shows that turnover years tend to be higher except for 2016

In [226]:
write_file("./visuals/number of maintenance request.html", fig.to_html())

### Look at Maintenance Cost Each Year

In [227]:
maintenance_cost = repairs.groupby(["Year", "Property"])["Amount"].sum().rename("cost").reset_index()
maintenance_cost

,Year,Property,cost
0,2015.0,bright leaf,657.00
1,2015.0,fort pike,360.00
2,2016.0,bright leaf,280.00
3,2016.0,fort pike,704.00
4,2017.0,bright leaf,909.99
5,2017.0,fort pike,240.00
6,2018.0,bright leaf,300.00
7,2018.0,fort pike,1550.50
8,2019.0,bright leaf,135.00
9,2019.0,fort pike,200.00


In [228]:
fig = px.line(maintenance_cost, x="Year", y="cost", color="Property", title="Cost of Maintenance by Year", color_discrete_map={
        'fort pike': 'blue',
        'bright leaf': 'green'
    })
fig

We can see that while 2016 and 2018 had the same number of maintenance request, in 2018 it cost more.

In [229]:
write_file("./visuals/maintenance cost by year.html", fig.to_html())

### Looking at Repairs and Maintenance as a proportion of Revenue

In [230]:
#Fort Pike
fp = df.loc[df["Property"]=="fort pike"]

fp_rev = fp.loc[fp["Split"]=="Rental Income"]
fp_rev = fp_rev.groupby("Year").sum().reset_index()

fp_rm = fp.loc[fp["Split"]=="Repairs and Maintenance"]
fp_rm = fp_rm.groupby("Year").sum().reset_index()
fp_rm["Amount"]= fp_rm["Amount"]*-1
print(fp_rm)

#Bright Leaf
bl = df.loc[df["Property"]=="bright leaf"]

bl_rev = bl.loc[bl["Split"]=="Rental Income"]
bl_rev = bl_rev.groupby("Year").sum().reset_index()

bl_rm = bl.loc[bl["Split"]=="Repairs and Maintenance"]
bl_rm = bl_rm.groupby("Year").sum().reset_index()
bl_rm["Amount"]= bl_rm["Amount"]*-1
bl_rm

     Year   Amount
0  2015.0   360.00
1  2016.0   704.00
2  2017.0   240.00
3  2018.0  1550.50
4  2019.0   200.00
5  2020.0  3029.27
6  2021.0  5061.70


,Year,Amount
0,2015.0,657.00
1,2016.0,280.00
2,2017.0,909.99
3,2018.0,300.00
4,2019.0,135.00
5,2020.0,1040.00
6,2021.0,85.00


In [231]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=("Fort Pike", "Bright Leaf")
)
#Fort Pike Graph
trace1 = go.Bar(x = fp_rev["Year"], y=fp_rev["Amount"], marker=dict(color="green", opacity=1), name="Revenue - Fort Pike")
trace2 = go.Bar(x = fp_rm["Year"], y=fp_rm["Amount"], marker=dict(color="red", opacity=1), name="Repairs and Maintenance - Fort Pike")
fig.add_trace(trace1, row=1, col=1)
fig.add_trace(trace2, row=1, col=1)
fig.update_xaxes(title_text="Year", row=1, col=1)
fig.update_yaxes(title_text="Amount",range=[0, 22000], row=1, col=1)

#Bright Leaf Graph
trace3 = go.Bar(x = bl_rev["Year"], y=bl_rev["Amount"], marker=dict(color="green", opacity=1), name="Revenue - Bright Leaf")
trace4 = go.Bar(x = bl_rm["Year"], y=bl_rm["Amount"], marker=dict(color="red", opacity=1), name="Repairs and Maintenance - Bright Leaf")
fig.add_trace(trace3, row=1, col=2)
fig.add_trace(trace4, row=1, col=2)
fig.update_xaxes(title_text="Year", row=1, col=2)
fig.update_yaxes(title_text="Amount", range=[0, 22000], row=1, col=2)

fig.update_layout(title = "Repairs and Maintenace as a Proportion of Revenue", barmode="overlay")

While previously, we saw that 2016 had the same amount of request as 2018, we can see that 2018 cost more in request.

In [232]:
write_file("./visuals/Repairs and Maintenance Rev.html", fig.to_html())

### Look at Net Operating Income (Revenue - Total Operating Expenses)

In [233]:
goi_fp = fp.loc[fp["Split"]=="Rental Income"].groupby("Year")["Amount"].sum() #Gross Operating Income
tox_fp = fp.loc[fp["Split"]!="Rental Income"].groupby("Year")["Amount"].sum()*-1 # Total Operating Expenses
noi_fp = (goi_fp - tox_fp).reset_index()
noi_fp

,Year,Amount
0,2015.0,15567.91
1,2016.0,14814.91
2,2017.0,15468.91
3,2018.0,12304.41
4,2019.0,17346.84
5,2020.0,13186.37
6,2021.0,12006.29


In [234]:
goi_bl = bl.loc[bl["Split"]=="Rental Income"].groupby("Year")["Amount"].sum() #Gross Operating Income
tox_bl = bl.loc[bl["Split"]!="Rental Income"].groupby("Year")["Amount"].sum()*-1 # Total Operating Expenses
noi_bl = (goi_bl - tox_bl).reset_index()
noi_bl

,Year,Amount
0,2015.0,10381.00
1,2016.0,10796.00
2,2017.0,9381.01
3,2018.0,10890.00
4,2019.0,9107.83
5,2020.0,10863.70
6,2021.0,12453.00


In [235]:
noi_fp = (goi_fp - tox_fp).reset_index().rename(columns = {"Amount":"Amount_FP"})
noi_fp

,Year,Amount_FP
0,2015.0,15567.91
1,2016.0,14814.91
2,2017.0,15468.91
3,2018.0,12304.41
4,2019.0,17346.84
5,2020.0,13186.37
6,2021.0,12006.29


In [236]:
noi_bl = (goi_bl - tox_bl).reset_index().rename(columns = {"Amount":"Amount_BL"})
noi_bl

,Year,Amount_BL
0,2015.0,10381.00
1,2016.0,10796.00
2,2017.0,9381.01
3,2018.0,10890.00
4,2019.0,9107.83
5,2020.0,10863.70
6,2021.0,12453.00


In [238]:
fig = make_subplots(
    rows=1, cols=1)
#Fort Pike Graph
trace1 = go.Bar(x = noi_fp["Year"], y=noi_fp["Amount_FP"], marker=dict(color="Blue", opacity=1), name="Fort Pike")
fig.add_trace(trace1)

#Bright Leaf Graph
trace2 = go.Bar(x = noi_bl["Year"], y=noi_bl["Amount_BL"], marker=dict(color="green", opacity=1), name="Bright Leaf")
fig.add_trace(trace2)

fig.update_xaxes(title_text="Year")
fig.update_yaxes(title_text="Amount")
fig.update_layout(title = "Net Operating Income", barmode="group")

In [239]:
write_file("./visuals/NOI.html", fig.to_html())

### Looking at Cap Rates (Net Operating Income / Market Value of Property)

In [240]:
#September Zestimate for each year
zestimate_fp = {
    "Year":[year for year in range(2015,2022+1)],
    "Zestimate_FP":[304200, 300400, 334600, 347100, 365300, 385800, 480400, 527500]
}
assumed_value_fp = pd.DataFrame(data=zestimate_fp)

In [241]:
#September Zestimate for each year
zestimate_bl = {
    "Year":[year for year in range(2015,2022+1)],
    "Zestimate_BL":[143300, 193400, 213300, 264600, 264100, 285600, 353300, 395900]
}
assumed_value_bl = pd.DataFrame(data=zestimate_bl)

In [242]:
#Changing the Year from float to int
noi_fp["Year"] = noi_fp["Year"].astype("int")
noi_bl["Year"] = noi_bl["Year"].astype("int")

In [243]:
#Cap Rate with NOI
cap_fp = noi_fp.merge(assumed_value_fp, on="Year", how="inner")
cap_fp["Cap_Rate_FP"] = (cap_fp["Amount_FP"]/cap_fp["Zestimate_FP"])*100
cap_fp

,Year,Amount_FP,Zestimate_FP,Cap_Rate_FP
0,2015,15567.91,304200,5.117656
1,2016,14814.91,300400,4.931728
2,2017,15468.91,334600,4.623105
3,2018,12304.41,347100,3.544918
4,2019,17346.84,365300,4.748656
5,2020,13186.37,385800,3.417929
6,2021,12006.29,480400,2.499228


In [244]:
#Cap Rate with NOI
cap_bl = noi_bl.merge(assumed_value_bl, on="Year", how="inner")
cap_bl["Cap_Rate_BL"] = (cap_bl["Amount_BL"]/cap_bl["Zestimate_BL"])*100
cap_bl

,Year,Amount_BL,Zestimate_BL,Cap_Rate_BL
0,2015,10381.00,143300,7.244243
1,2016,10796.00,193400,5.582213
2,2017,9381.01,213300,4.398036
3,2018,10890.00,264600,4.115646
4,2019,9107.83,264100,3.448629
5,2020,10863.70,285600,3.803817
6,2021,12453.00,353300,3.524766


In [245]:
#Cap Rate with no expenses Fort Pike
goi_fp_ = goi_fp.reset_index() 
cap_fp_ = goi_fp_.merge(assumed_value_fp, on="Year", how="inner")
cap_fp_["Cap_Rate_FP"] = (cap_fp_["Amount"]/cap_fp_["Zestimate_FP"])*100
cap_fp_

,Year,Amount,Zestimate_FP,Cap_Rate_FP
0,2015.0,18910.00,304200,6.216305
1,2016.0,18545.00,300400,6.173435
2,2017.0,18705.00,334600,5.590257
3,2018.0,16770.00,347100,4.831461
4,2019.0,20530.00,365300,5.620038
5,2020.0,21115.00,385800,5.473043
6,2021.0,20662.16,480400,4.301032


In [246]:
#Cap Rate with no expenses Bright Leaf
goi_bl_ = goi_bl.reset_index()
cap_bl_ = goi_bl_.merge(assumed_value_bl, on="Year", how="inner")
cap_bl_["Cap_Rate_BL"] = (cap_bl_["Amount"]/cap_bl_["Zestimate_BL"])*100
cap_bl_

,Year,Amount,Zestimate_BL,Cap_Rate_BL
0,2015.0,12800.0,143300,8.932310
1,2016.0,12975.0,193400,6.708893
2,2017.0,12595.0,213300,5.904829
3,2018.0,13140.0,264600,4.965986
4,2019.0,11280.0,264100,4.271109
5,2020.0,14304.2,285600,5.008473
6,2021.0,14940.0,353300,4.228701


In [247]:
fig = make_subplots(
    rows=1, cols=2, subplot_titles=("Cap Rate % using NOI", "Cap Rate % using Revenue"))
#Using NOI
trace1 = go.Bar(x = cap_fp["Year"], y=cap_fp["Cap_Rate_FP"], marker=dict(color="Blue", opacity=1), name="Fort Pike w/ NOI")
fig.add_trace(trace1, row=1, col=1)
trace2 = go.Bar(x = cap_bl["Year"], y=cap_bl["Cap_Rate_BL"], marker=dict(color="green", opacity=1), name="Bright Leaf w/ NOI")
fig.add_trace(trace2, row=1, col=1)

#Using Revenue
trace3 = go.Bar(x = cap_fp_["Year"], y=cap_fp_["Cap_Rate_FP"], marker=dict(color="Blue", opacity=1), name="Fort Pike w/ Revenue")
fig.add_trace(trace3, row=1, col=2)
trace4 = go.Bar(x = cap_bl_["Year"], y=cap_bl_["Cap_Rate_BL"], marker=dict(color="green", opacity=1), name="Bright Leaf w/ Revenue")
fig.add_trace(trace4, row=1, col=2)

fig.update_xaxes(title_text="Year", row = 1, col =1)
fig.update_yaxes(title_text="Cap Rate %", range=[0,20], row = 1, col = 1)
fig.update_xaxes(title_text="Year", row = 1, col =2)
fig.update_yaxes(title_text="Cap Rate %", range=[0,20], row = 1, col = 2)
fig.update_layout(title = "Cap Rate", barmode="group")

In [248]:
write_file("./visuals/CapRate.html", fig.to_html())